# Description of Script
This script creates the visit features using the pharmacy data

**Author:** Marang Mutloatse

**Version:** 0.0.1

**State:** Development

## Import Libraries

In [1]:
import os
# get parent user path
user_dev_path = os.path.dirname(os.getcwd())

# get parent path for package
package_path = os.path.dirname(user_dev_path)

In [2]:
import sys
sys.path.insert(0,package_path + '/src/')
import pandas as pd
import datetime as dt
import warnings
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import glob
import yaml
from hiv_support_package import visit_features,target_creation
from datetime import timedelta
plt.style.use('fivethirtyeight')
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

## Loading Data

In [3]:
try: 
    with open (user_dev_path + '/' + 'config_LTFU.yaml', 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print(f'Error reading the config file, {e}')

In [4]:
input_root = config['rise_files']['raw_data_path']
pharmacy_file = config['rise_files']['raw_pharmacy_file']
regimen_id_file = config['rise_files']['raw_regimen_file']
pharmacy_input = input_root + pharmacy_file
regimen_input = input_root + regimen_id_file
print(pharmacy_input)
print(regimen_input)

/data/rise_data/concat_pharmacy_data.csv
/data/rise_data/regimen.csv


### Working Functions

In [5]:
def load_excel_sheet(path: str):
    if path is not None:
        try:
            xls = pd.ExcelFile(path)
            df = xls.parse(skiprows=0)
            return df
        except Exception as e:
            print(f"Exception on loading excel spreadsheet with error: {e}")

def convert_to_datetime(df,date_col: str):
    if df is not None:
        try:
            df[date_col] = pd.to_datetime(df[date_col],errors='coerce')
            return df
        except Exception as e:
            print(f"Exception on converting to datetime with error: {e}") 
            
def print_num_rows_cols(df, df_name: str):
        if df is not None:
            return print(f"{df_name} data has {df.shape[1]} columns and {df.shape[0]} rows.") 

def check_null_values(df):
    if df is not None:
        try:
            missing_value = df.isnull().sum()
            mis_val_percent = 100 * df.isnull().sum() / len(df)
            mis_val_table = pd.concat([missing_value, mis_val_percent], axis=1)
            mis_val_table_rename_cols = mis_val_table.rename(
                columns={0: "Missing Values", 1: "% of Total Values"}
            )
            mis_val_table_rename_cols = (
                mis_val_table_rename_cols[mis_val_table_rename_cols.iloc[:, 1] != 0]
                .sort_values("% of Total Values", ascending=False)
                .round(1)
            )
            print(
                "There are "
                + str(mis_val_table_rename_cols.shape[0])
                + " columns that have missing values"
            )
            return mis_val_table_rename_cols
        except Exception as e:
            print(f"Exception on checking missing values with error: {e}")

## Import Data

In [6]:
%%time
df = pd.read_csv(pharmacy_input, low_memory=False)
df_regimen = pd.read_csv(regimen_input)

CPU times: user 7.5 s, sys: 1.04 s, total: 8.54 s
Wall time: 8.54 s


In [7]:
df_regimen.drop(columns=['REGIMEN_ID','COMPOSITION'],inplace=True)

## Filter For ART

In [8]:
ART_LIST = ['ART First Line Adult','ART Second Line Adult','ART First Line Children','ART Second Line Children','Third Line']
df = df[df['Regimen Line'].isin(ART_LIST)]

In [9]:
print("These are the counts for the different Regimen Lines after filtering:")
df['Regimen Line'].value_counts()

These are the counts for the different Regimen Lines after filtering:


ART First Line Adult        2490734
ART First Line Children       78773
ART Second Line Adult         41530
ART Second Line Children       2232
Third Line                      138
Name: Regimen Line, dtype: int64

## Merge RegimenID on Main Table

In [10]:
df = pd.merge(df,df_regimen,left_on="Regimen",right_on="DESCRIPTION",how="left")

In [11]:
start_date = '2015-01-01 00:00:00'
LAST_EXPECTED_VISIT = '2021-05-31 00:00:00' 

## Data Description and Verification

In [12]:
print_num_rows_cols(df,'Pharmacy Data')

Pharmacy Data data has 22 columns and 3744411 rows.


In [13]:
pd.set_option('display.max_columns', 500)
df = df.sort_values(by=['PATIENT_ID','DATE_VISIT'])

In [14]:
print("Shape before cleaning by date: ")
print_num_rows_cols(df,'Pharmacy')

date_cols = ['DATE_VISIT','NEXT_APPOINTMENT']

df['DATE_VISIT'] = pd.to_datetime(df['DATE_VISIT'], errors='coerce')
df['NEXT_APPOINTMENT'] = pd.to_datetime(df['NEXT_APPOINTMENT'], errors='coerce')
df['EF_MEDS_CONSUMED_DATE'] = df['DATE_VISIT']+pd.to_timedelta(df['DURATION'],unit='d')

df = df.loc[(df['DATE_VISIT'] >= '1998-01-01 00:00:00') & (df['DATE_VISIT'] <= '2021-05-31 00:00:00')]

print(" ")
print("Shape after limiting by date: ")
print_num_rows_cols(df,'Pharmacy')

Shape before cleaning by date: 
Pharmacy data has 22 columns and 3744411 rows.
 
Shape after limiting by date: 
Pharmacy data has 23 columns and 3662499 rows.


In [15]:
check_null_values(df)

There are 5 columns that have missing values


Missing Values  % of Total Values
ADR_IDS                  3662443              100.0
DMOC_TYPE                2896530               79.1
BODY_WEIGHT              1229425               33.6
ADR_SCREENED              713222               19.5
NEXT_APPOINTMENT           10671                0.3

In [16]:
df = df[~df['DATE_VISIT'].isnull()]
df_check = df.copy()
print("df with all next appointments:",df.shape)
df.loc[df['NEXT_APPOINTMENT'].isnull(),'NEXT_APPOINTMENT'] = df.DATE_VISIT+ pd.to_timedelta(df['DURATION'] - 3,unit='d')
print("df with NULL next appointments filled:",df.shape)

df with all next appointments: (3662499, 23)
df with NULL next appointments filled: (3662499, 23)


In [17]:
print("PATIENT_ID: " + "{:,}".format(df.PATIENT_ID.nunique()))

PATIENT_ID: 101,399


# Data Transformation

## Data Deduplication

In [18]:
print(f"Before deduplication the dataframe has {df.shape[0]} rows")
print(" ")
# df.drop_duplicates(subset=['PATIENT_ID','DATE_VISIT','NEXT_APPOINTMENT','Regimen','Regimen Line'],inplace=True)
df.drop_duplicates(subset=['PATIENT_ID','DATE_VISIT','NEXT_APPOINTMENT','DMOC_TYPE','DURATION','MORNING','AFTERNOON','EVENING','ADHERENCE'],inplace=True,keep="last")
print(f"After deduplication the dataframe has {df.shape[0]} rows")

Before deduplication the dataframe has 3662499 rows
 
After deduplication the dataframe has 1391869 rows


In [19]:
print(f"There are {df.loc[:, df.columns != 'PHARMACY_ID'].duplicated(subset=['PATIENT_ID','DATE_VISIT','NEXT_APPOINTMENT','DMOC_TYPE','DURATION','MORNING','AFTERNOON','EVENING','ADHERENCE']).sum()} duplicate rows based on the DATE_COLLECTED,PATIENT_ID,DATE_REPORTED and resultab field")

There are 0 duplicate rows based on the DATE_COLLECTED,PATIENT_ID,DATE_REPORTED and resultab field


In [20]:
print(f"Before secondary deduplication the dataframe has {df.shape[0]} rows")
print(" ")
df.drop_duplicates(subset=['PATIENT_ID','DATE_VISIT','NEXT_APPOINTMENT','Regimen'],inplace=True,keep="last")
print(f"After deduplication the dataframe has {df.shape[0]} rows")

Before secondary deduplication the dataframe has 1391869 rows
 
After deduplication the dataframe has 905231 rows


In [21]:
#df_check[df_check['PATIENT_ID']== 151336][['PATIENT_ID','DATE_VISIT','']]

In [22]:
# print(f"Before date (within 3 days) deduplication the dataframe has {df.shape[0]} rows")
# print(" ")
# df = df[(df.groupby('PATIENT_ID')
#       ['DATE_VISIT']
#       .diff(-1).dt.days.mul(-1) # calculate the difference
#       .fillna(float('inf'))     # make sure last row is kept
#       .ge(1)                    # select diff =2
#    )]
# print(f"After deduplication the dataframe has {df.shape[0]} rows")

## Stats on dates for LTFU target definition

In [23]:
 df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] = (df['NEXT_APPOINTMENT'] - df['DATE_VISIT']).dt.days

## Date Featurization

In [24]:
# Generate Date Features
df = visit_features.generate_date_features(df,'DATE_VISIT')
df = visit_features.generate_date_features(df,'NEXT_APPOINTMENT')
df = visit_features.generate_date_features(df,'EF_MEDS_CONSUMED_DATE')

Generating features for DATE_VISIT, starting with  rows:905,231; cols: 24
Generating features for DATE_VISIT,ending with  rows:905,231; cols: 36
Generating features for NEXT_APPOINTMENT, starting with  rows:905,231; cols: 36
Generating features for NEXT_APPOINTMENT,ending with  rows:905,231; cols: 48
Generating features for EF_MEDS_CONSUMED_DATE, starting with  rows:905,231; cols: 48
Generating features for EF_MEDS_CONSUMED_DATE,ending with  rows:905,231; cols: 60


In [25]:
# Get first visit day
df['EF_FIRST_DATE_VISIT'] = df.groupby('PATIENT_ID')['DATE_VISIT'].transform('min')

In [26]:
# generate sequence based off patient ID
df['EF_ID_SEQUENCE'] = df.groupby('PATIENT_ID').cumcount() + 1

In [27]:
## Regimen Change
m3 = df.groupby('PATIENT_ID')['Regimen Line'].shift().ne(df['Regimen Line'])
m4 = df['Regimen Line'].isin(["ART Second Line Adult", "ART Second Line Children","Third Line"])
df['REGIMEN_LINE_CHANGE'] = (m3 & m4).astype(int)

## Inclusion Criteria

In [28]:
df = df.loc[(df['EF_FIRST_DATE_VISIT'] > start_date)]
print("Shape after filtering early starters before: ", start_date, visit_features.format_shape(df))

Shape after filtering early starters before:  2015-01-01 00:00:00 rows:606,868; cols: 63


In [29]:
df['EF_MONTH_UNIQUE'] = df['EF_YEAR_DATE_VISIT'].apply(str)+'-'+df['EF_MONTH_DATE_VISIT'].apply(str)

## Days since last visit

In [30]:
df = visit_features.days_months_since_visit_features(df)
df = df.rename(columns={'NEXT_APPOINTMENT': 'EF_NEXT_APPOINTMENT'})
df = visit_features.next_actual_visit_features(df)
df = visit_features.months_first_visit_features(df)

### Outlier Filtration

# Target Creation

In [31]:
# Classic LTFU Pepfar Target
df = target_creation.missed_visit_target(df,LAST_EXPECTED_VISIT)

# General Targets
df = target_creation.next_visit_target(df,LAST_EXPECTED_VISIT)
df = target_creation.last_visit_target(df,LAST_EXPECTED_VISIT)

# Rise target
df = target_creation.rise_missed_visit_target(df,LAST_EXPECTED_VISIT)

## Other Visit Features

In [32]:
df = visit_features.days_late_features(df)
df = visit_features.meds_days_late_features(df)

In [33]:
# Indicator Based on appointment
df['EF_LATE_THIS_VISIT'] = df['EF_DAYS_LATE'].dt.days>0

# Indicator Based on meds
df['EF_LATE_THIS_VISIT_MEDS'] = df['EF_DAYS_LATE_MEDS'].dt.days>0

# Late Last visit features 
df = visit_features.late_last_visit_feature(df)

# Meds late last visit features
df = visit_features.meds_late_last_visit_feature(df)

In [34]:
dist = df['EF_DAYS_BETWEEN_NEXT_APPOINTMENT_AND_ACTUAL_VISIT_DATE'
  ].dt.days.sort_values().value_counts(normalize=True, bins=[-400,-20,0,30,90,3000]).sort_index()
print(dist)
print(df['EF_LATE_THIS_VISIT'].value_counts(normalize=True))

check = df['EF_DAYS_BETWEEN_MEDS_CONSUMED_AND_ACTUAL_VISIT_DATE'
  ].dt.days.sort_values().value_counts(normalize=True, bins=[-400,-20,0,30,90,3000]).sort_index()
print(check)
print(df['EF_LATE_THIS_VISIT_MEDS'].value_counts(normalize=True))

(-400.001, -20.0]    0.188983
(-20.0, 0.0]         0.177589
(0.0, 30.0]          0.149921
(30.0, 90.0]         0.076704
(90.0, 3000.0]       0.187685
Name: EF_DAYS_BETWEEN_NEXT_APPOINTMENT_AND_ACTUAL_VISIT_DATE, dtype: float64
False    0.585656
True     0.414344
Name: EF_LATE_THIS_VISIT, dtype: float64
(-400.001, -20.0]    0.208147
(-20.0, 0.0]         0.161437
(0.0, 30.0]          0.173586
(30.0, 90.0]         0.073687
(90.0, 3000.0]       0.166133
Name: EF_DAYS_BETWEEN_MEDS_CONSUMED_AND_ACTUAL_VISIT_DATE, dtype: float64
False    0.586594
True     0.413406
Name: EF_LATE_THIS_VISIT_MEDS, dtype: float64


In [35]:
# Based on next appointment date
df = visit_features.late_days_count_ratio_features(df, 3,'PATIENT_ID')
df = visit_features.late_days_count_ratio_features(df, 28,'PATIENT_ID')
df = visit_features.late_days_count_ratio_features(df, 60,'PATIENT_ID')
df = visit_features.late_days_count_ratio_features(df, 90,'PATIENT_ID')

In [36]:
# Based on medication consumed date
df = visit_features.meds_late_days_count_ratio_features(df, 3,'PATIENT_ID')
df = visit_features.meds_late_days_count_ratio_features(df, 28,'PATIENT_ID')
df = visit_features.meds_late_days_count_ratio_features(df, 60,'PATIENT_ID')
df = visit_features.meds_late_days_count_ratio_features(df, 90,'PATIENT_ID')

### Facility Features

In [37]:
df['EF_FACILITY_ID'] = df['FACILITY_ID']
df = visit_features.create_last_facility_codes_EF(df,'PATIENT_ID')

In [38]:
# Counts number of Facilties
df['EF_VISITS_AT_THIS_FACILITY'] = df.groupby(['PATIENT_ID','FACILITY_ID'])['PHARMACY_ID'].cumcount()+1
df['EF_FACILITY_COMPOSITION'] = (df['EF_VISITS_AT_THIS_FACILITY']/df['EF_ID_SEQUENCE']).round(1)

# Regimen count on current regimen
df['EF_VISITS_ON_THIS_REGIMEN'] = df.groupby(['PATIENT_ID','Regimen'])['PHARMACY_ID'].cumcount()+1
df['EF_REGIMEN_COMPOSITION'] = (df['EF_VISITS_ON_THIS_REGIMEN']/df['EF_ID_SEQUENCE']).round(1)

In [39]:
# # Calculate the number of facilties visited
# df['EF_NUM_FACILTIES_VISITED'] = df.drop_duplicates(subset=['PATIENT_ID', 'FACILITY_ID']
#   ).groupby('PATIENT_ID').cumcount() + 1

# df['EF_NUM_FACILTIES_VISITED'] = df['EF_NUM_FACILTIES_VISITED'].fillna(method='ffill',inplace=True)

In [40]:
df = df.rename(columns={'EF_DAYOFMONTH_DATE_VISIT': 'EF_DAY_OF_MONTH_DATE_VISIT', 
                        'EF_DAYOFWEEK_DATE_VISIT': 'EF_DAY_OF_WEEK_DATE_VISIT'})

## Convert Boolean Columns to Numeric

In [41]:
bool_col = df.select_dtypes(include='bool').columns.to_list()
for col in bool_col:
    df[col] = df[col].astype('int')

In [42]:
# df['EF_BIN_DURATION'] = pd.qcut(df['DURATION'], 5, 
#                                labels = ['very low', 'low', 'medium', 
#                                         'high', 'very high'])

# df['EF_BIN_DURATION'] = pd

In [43]:
adr_screened_dict = {"No":0,"Yes":1,"?":0}
df['ADR_SCREENED'] = df['ADR_SCREENED'].map(adr_screened_dict)
df['ADR_SCREENED'] = df['ADR_SCREENED'].fillna(0)

In [44]:
# Convert to Days Field to Integer
df['EF_DAYS_BETWEEN_NEXT_APPOINTMENT_AND_ACTUAL_VISIT_DATE'] = df['EF_DAYS_BETWEEN_NEXT_APPOINTMENT_AND_ACTUAL_VISIT_DATE']/np.timedelta64(1,'D')
df['EF_DAYS_BETWEEN_MEDS_CONSUMED_AND_ACTUAL_VISIT_DATE'] = df['EF_DAYS_BETWEEN_MEDS_CONSUMED_AND_ACTUAL_VISIT_DATE']/np.timedelta64(1,'D')
df['EF_DAYS_LATE'] = df['EF_DAYS_LATE']/np.timedelta64(1,'D')
df['EF_DAYS_LATE_MEDS'] = df['EF_DAYS_LATE_MEDS']/np.timedelta64(1,'D')

## One Hot Encoding

In [45]:
# drop in final data to avodi dummy variable trap
def one_hot_encode_col(df,dummies_col:str,prefix_entry:str):
    dummies = pd.get_dummies(df[dummies_col], prefix=prefix_entry,drop_first=False)
    df = df.join(dummies)
    return df

In [46]:
# Otherise data
allowed_vals = df['Regimen'].value_counts(normalize=True).head(15).index.tolist()
df.loc[~df["Regimen"].isin(allowed_vals), "Regimen"] = "Other"

In [47]:
df = one_hot_encode_col(df,'State','EF_STATE')
#df = one_hot_encode_col(df,'L.G.A','EF_LGA')
df = one_hot_encode_col(df,'Regimen','EF_REGIMEN')
df = one_hot_encode_col(df,'Regimen Line','EF_REGIMEN_LINE')
df = one_hot_encode_col(df,'REGIMENTYPE_ID',"EF_REGIMEN_LINE_ID")
#df = one_hot_encode_col(df,'Facility Name','EF_FACILITY_NAME')

# Export Columns Preparation

In [48]:
# Replace spaces in columns names
df.columns = df.columns.str.replace(' ','_')

# Standardizing columns
df.columns = map(lambda x: str(x).upper(), df.columns)

In [49]:
# drop columns
drop_cols = [
    "MORNING",
    "AFTERNOON",
    "EVENING",
    "ADR_IDS",
    "DMOC_TYPE",
    "BODY_WEIGHT",
    "DESCRIPTION",
    "STATE",
    "REGIMENTYPE_ID",
    "REGIMEN",
    "REGIMEN_LINE",
    #"L.G.A",
]

df.drop(columns = drop_cols,inplace=True)

In [50]:
df = df.add_prefix('V_')

In [51]:
df.to_csv(input_root+'feature_pharmacy.csv',index=False,header=True)